In [1]:
 from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers

In [3]:
# 可以直接在seq中给予层的列表，以及指定模型的名称(keras中绝大多数对象（层）都是有名称的）

model = keras.models.Sequential(
    layers=[
            keras.layers.Dense(2, activation="relu", name="layer1"),
            keras.layers.Dense(3, activation="relu", name="layer2"),
            keras.layers.Dense(4, name="layer3"),
           ],
    name='my_seq_model'
                                )

x = keras.backend.ones((3, 3), dtype='int32', name='input')
print(x)
y = model(x)


# 以上这种等价于

layer1 = layers.Dense(2, activation="relu", name="layer1")
layer2 = layers.Dense(3, activation="relu", name="layer2")
layer3 = layers.Dense(4, name="layer3")

x_1 = tf.ones((3, 3))
print(x)
y_1 = layer3(layer2(layer1(x)))
print(y_1)

<tf.Variable 'input:0' shape=(3, 3) dtype=int32, numpy=
array([[1, 1, 1],
       [1, 1, 1],
       [1, 1, 1]])>
<tf.Variable 'input:0' shape=(3, 3) dtype=int32, numpy=
array([[1, 1, 1],
       [1, 1, 1],
       [1, 1, 1]])>
tf.Tensor(
[[ 0.9060266  -0.01674981  0.2985723   0.33688113]
 [ 0.9060266  -0.01674981  0.2985723   0.33688113]
 [ 0.9060266  -0.01674981  0.2985723   0.33688113]], shape=(3, 4), dtype=float32)


In [5]:
# 任何模型都可以访问子层
layers_lis = model.layers
model.get_layer(index=0, name='dense_1')  # 该函数底层代码本质也是取layers属性，只不过可以通过index和name来查找
print(layers_lis)

# 除了可以通过add添加层，还可以通过pop删，但是从尾部开始
model.pop()
print(len(model.layers))

[<tensorflow.python.keras.layers.core.Dense object at 0x000001DD88CDC648>, <tensorflow.python.keras.layers.core.Dense object at 0x000001DD88CDC9C8>, <tensorflow.python.keras.layers.core.Dense object at 0x000001DD88CDCDC8>]
2


In [9]:
# 初始化后层的权重都是空的，只有在输入时，会初始化

layer_1 = keras.layers.Dense(units=3)
print(layer_1.weights)

x = keras.backend.ones((1, 4))
y = layer_1(x)
print(layer_1.weights)

# 对于没有指定输入形状的模型，是无weights和summary的

model = keras.models.Sequential(layers=[keras.layers.Dense(6),
                                        keras.layers.Dense(10)],
                                name='ceshi_model')
print(model.summary())  # 都会报错，因为无具体形状，模型无法构建
print(model.weights)
# 故应最初add一个Input对象来启动模型，或者用第一层的input_shape来代替


[]
[<tf.Variable 'dense_5/kernel:0' shape=(4, 3) dtype=float32, numpy=
array([[ 0.5883374 ,  0.49605572, -0.7962869 ],
       [ 0.81412864, -0.5029481 ,  0.12368977],
       [-0.14361775, -0.6918834 ,  0.7993928 ],
       [-0.8522146 ,  0.1745404 , -0.21200162]], dtype=float32)>, <tf.Variable 'dense_5/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]


ValueError: Weights for model ceshi_model have not yet been created. Weights are created when the Model is first called on inputs or `build()` is called with an `input_shape`.

In [18]:
# 一旦构建成功，每层都会有input和output属性，可以调用
# 这样可以方便我们进行特征提取

initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)
print(initial_model.input)  # 把模型当层来对待，来调用输入属性
print(initial_model.inputs)  # 模型输入的列表
print(initial_model.layers[1].input)  # 对单个层，只有input而无inputs属性
print(initial_model.layers[1].output)

# 除此之外，每层都有一个可以冻结它的控制属性
# 最爱常用于迁移学习时，需要冻结base，训练顶层
initial_model.layers[1].trainable = False

Tensor("input_9:0", shape=(None, 250, 250, 3), dtype=float32)
[<tf.Tensor 'input_9:0' shape=(None, 250, 250, 3) dtype=float32>]
Tensor("conv2d_24/Relu:0", shape=(None, 123, 123, 32), dtype=float32)
Tensor("conv2d_25/Relu:0", shape=(None, 121, 121, 32), dtype=float32)
